In [21]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [22]:
robot.getStates()

{'q': array([-1.05593248e-04, -7.84475965e-01,  3.56295220e-03, -2.50966895e+00,
        -1.06560871e-03,  1.57109323e+00,  7.86168736e-01]),
 'dq': array([ 2.44816006e-03,  1.46172725e-03, -4.95307038e-04,  6.44718811e-05,
        -6.61234049e-04, -6.89224474e-04,  7.10209487e-04]),
 'T': array([-8.63957033e-03, -4.55194235e+00, -9.02548194e-01,  2.34527931e+01,
         1.03654706e+00,  1.92899299e+00, -7.64214694e-02])}

In [23]:
import numpy as np
from FR3Py.robot.model_collision_avoidance import PinocchioModel
import time

pin_robot = PinocchioModel()
joint_lb = np.array([-2.3093, -1.5133, -2.4937, -2.7478, -2.48, 0.8521, -2.6895])
joint_ub = np.array([2.3093, 1.5133, 2.4937, -0.4461, 2.48, 4.2094, 2.6895])
n_joints = 7
q_bar = 0.5*(joint_ub + joint_lb)
P_EE_desired = np.array([0.42, 0.50, 0.00])
R_EE_desired = np.array([[1, 0, 0],
                        [0, -1, 0],
                        [0, 0, -1]])
P_EE_initial = np.array([0.30, 0.0, 0.47])

### Inverse dynamic control (initial pose to pre-cleaning pose)

In [24]:
from cores.utils.trajectory_utils import TrapezoidalTrajectory

via_points = np.array([P_EE_initial, P_EE_desired])
target_time = np.array([0, 10])
traj = TrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=0.01)

In [25]:
from cores.utils.control_utils import get_torque_to_track_traj_const_ori

T = 12

Kp_task = np.diag([40,40,40,100,100,100])
Kd_task = np.diag([40,40,40,100,100,100])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])*10
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*10

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint
    u = np.clip(u_nominal, -20, 20)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))


### Setup the CBF

In [40]:
from cores.utils.proxsuite_utils import init_proxsuite_qp
import diffOptHelper as doh
from FR3Py.robot.model_collision_avoidance import BoundingShapeCoef
from scipy.spatial.transform import Rotation
from cores.utils.rotation_utils import get_quat_from_rot_matrix, get_Q_matrix_from_quat, get_dQ_matrix

# Load the obstacle
obs_pos_np = np.array([0.22, 0.5, 0.0])
obs_quat_np = np.array([0.0, 0.0, 0.0, 1.0]) # (qx, qy, qz, qw)
obs_size_np = np.array([0.05, 0.05, 0.05])
obs_coef_np = np.diag(1/obs_size_np**2)
obs_R_np = Rotation.from_quat(obs_quat_np).as_matrix()
obs_coef_np = obs_R_np @ obs_coef_np @ obs_R_np.T

# Load the bounding shape coefficients
BB_coefs = BoundingShapeCoef()

# CBF parameters
alpha0 = [1.3, 1.3, 1.3]
gamma1 = [3.0, 3.0, 3.0]
gamma2 = [10.0, 10.0, 10.0]
compensation = [0, 0, 0]
selected_BBs = ["HAND_BB", "LINK7_BB", "LINK6_BB"]

# Define proxuite problem
print("==> Define proxuite problem")
n_CBF = len(selected_BBs)
cbf_qp = init_proxsuite_qp(n_v=n_joints, n_eq=0, n_in=n_joints+n_CBF)


==> Define proxuite problem


### Press against the surface while tracking a circular trajectory

In [41]:
from cores.utils.trajectory_utils import CircularTrajectory
duration = 100
P_start_point = np.array([0.42, 0.50, 0.0])
P_center = np.array([0.32, 0.50, 0.0])
nominal_linear_vel = 0.05
R_b_to_w = np.eye(3)
traj = CircularTrajectory(P_center, P_start_point, nominal_linear_vel, R_b_to_w, duration)


In [42]:
T = 60
input_torque_lb = np.array([-87, -87, -87, -87, -87, -87, -87])
input_torque_ub = np.array([87, 87, 87, 87, 87, 87, 87])

cbf_active = True

Kp_task = np.diag([40,40,40,60,60,100])
Kd_task = np.diag([30,30,30,20,20,75])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])*200
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*160

F_ext = np.array([0, 0, -10, 0, 0, 0])

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Second objective: apply a force on the z axis to press the end-effector against the table
    u_press = J_EE.T @ F_ext

    # Third objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  nle + Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint + u_press

    if cbf_active:
        # Matrices for the CBF-QP constraints
        C = np.zeros([n_joints+n_CBF, n_joints])
        lb = np.zeros(n_joints+n_CBF)
        ub = np.zeros(n_joints+n_CBF)
        CBF_tmp = np.zeros(n_CBF)
        phi1_tmp = np.zeros(n_CBF)
        phi2_tmp = np.zeros(n_CBF)

        for kk in range(len(selected_BBs)):
            name_BB = selected_BBs[kk]
            P_BB = pin_info["P_"+name_BB]
            R_BB = pin_info["R_"+name_BB]
            J_BB = pin_info["J_"+name_BB][:,0:n_joints]
            dJdq_BB = pin_info["dJdq_"+name_BB]
            v_BB = J_BB @ dq
            D_BB = BB_coefs.coefs[name_BB]
            quat_BB = get_quat_from_rot_matrix(R_BB)
            alpha, _, alpha_dx_tmp, alpha_dxdx_tmp = doh.getGradientAndHessianEllipsoids(P_BB, quat_BB, D_BB, 
                            R_BB, obs_coef_np, obs_pos_np)
            
            # Order of parameters in alpha_dx_tmp and alpha_dxdx_tmp: [qx, qy, qz, qw, x, y, z]
            # Convert to the order of [x, y, z, qx, qy, qz, qw]
            alpha_dx = np.zeros(7)
            alpha_dx[0:3] = alpha_dx_tmp[4:7]
            alpha_dx[3:7] = alpha_dx_tmp[0:4]

            alpha_dxdx = np.zeros((7, 7))
            alpha_dxdx[0:3,0:3] = alpha_dxdx_tmp[4:7,4:7]
            alpha_dxdx[3:7,3:7] = alpha_dxdx_tmp[0:4,0:4]
            alpha_dxdx[0:3,3:7] = alpha_dxdx_tmp[4:7,0:4]
            alpha_dxdx[3:7,0:3] = alpha_dxdx_tmp[0:4,4:7]

            # CBF-QP constraints
            dx = np.zeros(7)
            dx[0:3] = v_BB[0:3]
            Q = get_Q_matrix_from_quat(quat_BB) # shape (4,3)
            dquat = 0.5 * Q @ v_BB[3:6] # shape (4,)
            dx[3:7] = dquat 
            dCBF =  alpha_dx @ dx # scalar
            CBF = alpha - alpha0[kk]
            phi1 = dCBF + gamma1[kk] * CBF

            dQ = get_dQ_matrix(dquat) # shape (4,3)
            tmp_vec = np.zeros(7)
            tmp_vec[3:7] = 0.5 * dQ @ v_BB[3:6] # shape (4,)
            tmp_mat = np.zeros((7,6))
            tmp_mat[0:3,0:3] = np.eye(3)
            tmp_mat[3:7,3:6] = 0.5 * Q

            C[kk,:] = alpha_dx @ tmp_mat @ J_BB @ Minv
            lb[kk] = - gamma2[kk]*phi1 - gamma1[kk]*dCBF - dx.T @ alpha_dxdx @ dx - alpha_dx @ tmp_vec \
                    - alpha_dx @ tmp_mat @ dJdq_BB + alpha_dx @ tmp_mat @ J_BB @ Minv @ (nle) + compensation[kk]
            ub[kk] = np.inf

            CBF_tmp[kk] = CBF
            phi1_tmp[kk] = phi1

        # CBF-QP constraints
        g = -u_nominal
        C[n_CBF:n_CBF+n_joints,:] = np.eye(n_joints)
        lb[n_CBF:] = input_torque_lb
        ub[n_CBF:] = input_torque_ub
        cbf_qp.update(g=g, C=C, l=lb, u=ub)
        cbf_qp.solve()
        u = cbf_qp.results.x
        for kk in range(n_CBF):
            phi2_tmp[kk] = C[kk,:] @ u - lb[kk]

    else:
        u = u_nominal
    
    u = u - nle
    u = np.clip(u, -10, 10)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))

### Lift from the whiteboard

In [106]:
via_points = np.array([P_EE, np.array([P_EE[0], P_EE[1], P_EE[2]+0.2])])
target_time = np.array([0, 5])
traj = TrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=0.01)

In [107]:
T = 7

Kp_task = np.diag([40,40,40,100,100,100])
Kd_task = np.diag([40,40,40,100,100,100])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])*10
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*10

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint
    u = np.clip(u_nominal, -20, 20)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))

TypeError: 'NoneType' object is not subscriptable